In [19]:
# Load a full run (model + predictor + loaders + splits) for evaluation
import os, sys, torch
from torch_geometric.transforms import ToUndirected

PROJECT_ROOT = os.path.abspath("..")  # if notebook is in notebooks/
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.load_graph_data import load_hetero_pt
from src.model_io import load_full_run
# === Load data and identify saved runs ===

from pathlib import Path
from torch_geometric.transforms import ToUndirected

# Add project root to path if needed
PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Imports from your project
from src.load_graph_data import load_hetero_pt

# Select device
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    os.environ.setdefault('PYTORCH_ENABLE_MPS_FALLBACK', '1')
    DEVICE = torch.device("cpu")
print('Using device:', DEVICE)

# === Load and preprocess graph ===

data = load_hetero_pt()
data = ToUndirected()(data)

# Optional: drop unused nodes/edges
keep = ['email', 'sender', 'url', 'domain', 'stem', 'email_domain', 'receiver']
keep_set = set(keep)

for ntype in list(data.node_types):
    if ntype not in keep_set:
        del data[ntype]

for et in list(data.edge_types):
    src, rel, dst = et
    if src not in keep_set or dst not in keep_set:
        del data[et]




Using device: cpu


/Users/mcandersyo/ITU/Research Project/GNN-Campaign-Detection/core/GNN/src/load_graph_data.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path, map_l

In [20]:
from pathlib import Path

def list_saved_runs(models_dir="../models"):
    """
    Returns a list of all run directories (e.g., 'run-2025-12-09_18-42-00') under the models folder.
    """
    models_path = Path(models_dir)
    run_dirs = sorted([p for p in models_path.glob("run-*") if p.is_dir()])
    print(f"Found {len(run_dirs)} saved run(s):")
    for run in run_dirs:
        print(" -", run.name)
    return run_dirs

def list_saved_models(run_name, models_dir="../models"):
    """
    Given a run name (e.g., 'run-2025-12-09_18-42-00'), returns all model checkpoint files under that run folder.
    """
    run_path = Path(models_dir) / run_name
    if not run_path.exists():
        raise FileNotFoundError(f"Run directory not found: {run_path}")
    model_paths = sorted(run_path.glob("*.pt"))
    print(f"Found {len(model_paths)} model(s) in {run_name}:")
    for m in model_paths:
        print(" -", m.name)
    return model_paths




In [21]:
from pathlib import Path
import torch
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

# Load the vote file (already uploaded earlier)
vote_file_path = "pair_votes_all.csv"
votes_df = pd.read_csv(vote_file_path)

# Extract email pairs and binary label indicating same campaign or not
def extract_ground_truth_pairs(df: pd.DataFrame):
    pairs = []
    for _, row in df.iterrows():
        a = row["email_a_id"]
        b = row["email_b_id"]
        same_campaign = row["vote"] == "same_campaign"
        pairs.append((a, b, same_campaign))
    return pairs

# Run extraction
ground_truth_pairs = extract_ground_truth_pairs(votes_df)
print(ground_truth_pairs[:5])  # Show a sample of the result


def compute_pairwise_clustering_metrics(ground_truth_pairs, cluster_labels):
    """
    Computes precision, recall, F1, and homogeneity from clustering vs. ground truth email campaign pairs.

    Parameters:
    - ground_truth_pairs: list of (email_a_id, email_b_id, same_campaign)
    - cluster_labels: dict {email_id: cluster_id}

    Returns:
    - dict with precision, recall, f1, and homogeneity
    """
    tp = fp = fn = tn = 0

    for email_a, email_b, same_campaign in ground_truth_pairs:
        if email_a not in cluster_labels or email_b not in cluster_labels:
            continue  # skip unknown nodes

        same_cluster = cluster_labels[email_a] == cluster_labels[email_b]

        if same_cluster and same_campaign:
            tp += 1
        elif same_cluster and not same_campaign:
            fp += 1
        elif not same_cluster and same_campaign:
            fn += 1
        else:
            tn += 1

    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall    = tp / (tp + fn) if (tp + fn) else 0.0
    f1        = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
    homogeneity = (tp + tn) / (tp + fp + fn + tn) if (tp + fp + fn + tn) else 0.0

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "homogeneity": homogeneity,
        "tp": tp,
        "fp": fp,
        "fn": fn,
        "tn": tn
    }

# -- Extract email embeddings from a model
@torch.no_grad()
def extract_email_embeddings(model, data, device):
    model.eval()
    x_dict = data.to(device).x_dict
    edge_index_dict = data.to(device).edge_index_dict
    h = model(x_dict, edge_index_dict)
    email_vecs = h['email'].cpu().numpy()
    email_ids = np.arange(len(email_vecs))  # Assumes IDs are ordered
    return email_vecs, email_ids

# -- Run HDBSCAN clustering and compute metrics
def compute_clustering_metrics(embeddings, labels):
    mask = labels != -1
    if mask.sum() > 1:
        silhouette = silhouette_score(embeddings[mask], labels[mask])
        db_index = davies_bouldin_score(embeddings[mask], labels[mask])
        ch_index = calinski_harabasz_score(embeddings[mask], labels[mask])
    else:
        silhouette, db_index, ch_index = -1, float('inf'), 0
    return silhouette, db_index, ch_index

# -- Main clustering analysis loop
from pathlib import Path

def run_clustering_analysis_across_models(data, device, run_dir, epsilon_values, ground_truth_pairs):
    from src.model_io import load_model_checkpoint

    run_path = Path("../models") / run_dir
    model_files = sorted(run_path.glob("*.pt"))

    results = []
    for eps in epsilon_values:
        print(f"\n### Clustering with epsilon={eps} ###")
        for model_file in model_files:
            print(f"Evaluating {model_file.name}...")
            model, predictor, checkpoint = load_model_checkpoint(device=device, metadata=data.metadata(), filename=model_file)

            embeddings, email_ids = extract_email_embeddings(model, data, device)

            clusterer = DBSCAN(eps=eps, min_samples=5, metric='euclidean')
            labels = clusterer.fit_predict(embeddings)

            silhouette, db_index, ch_index = compute_clustering_metrics(embeddings, labels)

            # Build a label map for fast lookup
            label_map = dict(zip(email_ids, labels))
            pairwise_metrics = compute_pairwise_clustering_metrics(ground_truth_pairs, label_map)

            results.append({
                "model_file": model_file.name,
                "epsilon": eps,
                "silhouette": silhouette,
                "db_index": db_index,
                "ch_index": ch_index,
                "n_clusters": len(set(labels)) - (1 if -1 in labels else 0),
                "n_noise": (labels == -1).sum(),
                "homogeneity": pairwise_metrics["homogeneity"],
                "precision": pairwise_metrics["precision"],
                "recall": pairwise_metrics["recall"],
                "f1": pairwise_metrics["f1"],
                "tp": pairwise_metrics["tp"],
                "fp": pairwise_metrics["fp"],
                "fn": pairwise_metrics["fn"],
                "tn": pairwise_metrics["tn"],
            })

    return results



[(4080, 4107, True), (1985, 3129, False), (2673, 4269, True), (3129, 5945, False), (7838, 785, True)]


In [22]:
epsilon_values =  [0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.08, 0.09]

from pathlib import Path
import pandas as pd

# --- Configuration ---
run_name = "run-2025-12-10_11-55-48"  # Replace with your actual run folder name
pair_votes_path = "pair_votes_all.csv"  # File next to notebook

votes_df = pd.read_csv(pair_votes_path)

# --- Extract ground-truth email pair labels ---
ground_truth_pairs = extract_ground_truth_pairs(votes_df)

for epsilon in epsilon_values:
    print(f"Running clustering analysis for {run_name} with ε = {epsilon}...")
    results = run_clustering_analysis_across_models(
        data=data,
        device=DEVICE,
        run_dir=run_name,
        epsilon_values=[epsilon],
        ground_truth_pairs=ground_truth_pairs,
    )

    # --- Display results ---
    results_df = pd.DataFrame(results)
    print("\nClustering results for ε =", epsilon)
    #display(results_df)

    # --- Optional: save to CSV inside run folder ---
    results_path = Path("../models") / run_name / f"clustering_results_eps_{str(epsilon).replace('.', '_')}.csv"
    results_df.to_csv(results_path, index=False)
    print(f"Saved results to {results_path}")



Running clustering analysis for run-2025-12-10_11-55-48 with ε = 0.02...

### Clustering with epsilon=0.02 ###
Evaluating best_model.pt...


/Users/mcandersyo/ITU/Research Project/GNN-Campaign-Detection/core/GNN/src/model_io.py:125: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, 

Evaluating model_epoch_1.pt...


/Users/mcandersyo/ITU/Research Project/GNN-Campaign-Detection/core/GNN/src/model_io.py:125: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, 

Evaluating model_epoch_10.pt...


...

Evaluating model_epoch_5.pt...


/Users/mcandersyo/ITU/Research Project/GNN-Campaign-Detection/core/GNN/src/model_io.py:125: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, 

Evaluating model_epoch_50.pt...


/Users/mcandersyo/ITU/Research Project/GNN-Campaign-Detection/core/GNN/src/model_io.py:125: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, 


Clustering results for ε = 0.09
Saved results to ../models/run-2025-12-10_11-55-48/clustering_results_eps_0_09.csv
